## XGBデフォルト

In [1]:
import numpy as np
import pandas as pd

import xgboost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk2')

In [3]:
train_pkl.shape

(55175, 12)

## 訓練データとテストデータに分割

In [4]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [5]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, 1:-1]
train_y = train.y

## 訓練データで訓練

In [6]:
XGB = XGBRegressor()

In [7]:
XGB = XGB.fit(train_X, train_y)

[22:14:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [8]:
XGB.feature_importances_

array([0.16900937, 0.16775638, 0.4275273 , 0.01268174, 0.04429195,
       0.01062545, 0.00591433, 0.00937364, 0.1025395 , 0.05028046],
      dtype=float32)

In [9]:
sorted(
    zip(map(lambda x: round(x, 3), XGB.feature_importances_), train.iloc[:, 1:].columns),
    reverse=True)

[(0.428, 'bedrooms'),
 (0.169, 'accommodates'),
 (0.168, 'bathrooms'),
 (0.103, 'number_of_reviews'),
 (0.05, 'review_scores_rating'),
 (0.044, 'cleaning_fee'),
 (0.013, 'beds'),
 (0.011, 'host_has_profile_pic'),
 (0.009, 'instant_bookable'),
 (0.006, 'host_identity_verified')]

## テストデータで実行

In [10]:
# ターゲットと特徴量の分割
test_x = test.iloc[:, 1:-1]
test_y = test.y

In [11]:
test_x.shape, test_y.shape

((11035, 10), (11035,))

In [12]:
pred_y = XGB.predict(test_x)

In [13]:
np.sqrt(mean_squared_error(test_y, pred_y))

132.66054363055545

## 検証データで実行

In [14]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk2')

In [15]:
valid.shape

(18528, 11)

In [16]:
# ID の保存
valid_pass = valid.id.values

In [17]:
valid_X = valid.iloc[:, 1:]

In [18]:
valid_X.shape, train_X.shape

((18528, 10), (44140, 10))

In [19]:
pred_valid_y = XGB.predict(valid_X)

In [20]:
pred_valid_y.shape

(18528,)

In [21]:
type(valid_pass), type(pred_valid_y)

(numpy.ndarray, numpy.ndarray)

In [22]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [23]:
result_df.to_csv("./XGB_1.csv", header=False)